In [1]:
# [Remi, Ojo]
# [5834990]
# [MMA]
# [2021W]
# [MMA869]
# [July 11, 2020]

## Sentiment Analysis

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re
import random


In [3]:
product=pd.read_csv("C:/Users/8oo/OneDrive/Documents/MMA 865 - Big Data Analytics/Assignment 1/sentiment_train.csv")
product_test=pd.read_csv("C:/Users/8oo/OneDrive/Documents/MMA 865 - Big Data Analytics/Assignment 1/sentiment_test.csv")
product.info()

product.info()

product.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2202 entries, 0 to 2201
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  2202 non-null   object
 1   Polarity  2202 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 34.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2202 entries, 0 to 2201
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  2202 non-null   object
 1   Polarity  2202 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 34.5+ KB


,Sentence,Polarity
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
#Checking the Balance of the dataset specific to polarity
product.Polarity.value_counts()

0    1113
1    1089
Name: Polarity, dtype: int64

## Data Cleaning // Preprocessing

In [5]:
import unidecode
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer

stop_words = set(stopwords.words('english'))

lemmer = WordNetLemmatizer()

def preprocess(x):
    x = x.lower()
    
    x = re.sub(r'[^\w\s]', '', x)
    
    x = unidecode.unidecode(x)
    
    x = re.sub(r'\d+', '', x)
    
    x = [lemmer.lemmatize(w) for w in x.split() if w not in stop_words]

    return ' '.join(x)

sentiment_train_clean = product['Sentence'].apply(preprocess)
sentiment_test_clean = product_test['Sentence'].apply(preprocess)

In [6]:
sentiment_train_clean.head()

0                                      wow loved place
1                                           crust good
2                                  tasty texture nasty
3    stopped late may bank holiday rick steve recom...
4                           selection menu great price
Name: Sentence, dtype: object

In [9]:
# Vectorize the data

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer(binary=True)
cv = cv.fit(sentiment_train_clean)
X = cv.transform(sentiment_train_clean)
X_test = cv.transform(sentiment_test_clean)

##

no_features = 2000

tf_vectorizer = CountVectorizer(min_df=0.05, max_df=0.5, max_features=no_features, ngram_range=[1,3])
tf_vectorizer = tf_vectorizer.fit_transform(sentiment_train_clean)

  (0, 1170)	1
  (0, 1347)	1
  (0, 1846)	1
  (0, 2767)	1
  (0, 3236)	1
  (0, 3586)	1
  (1, 1074)	1
  (1, 1170)	1
  (1, 1186)	1
  (1, 1679)	1
  (1, 1882)	1
  (1, 2266)	1
  (1, 3196)	1
  (2, 552)	1
  (2, 1347)	1
  (2, 1520)	1
  (2, 1747)	1
  (2, 2339)	1
  (2, 2514)	1
  (3, 433)	1
  (3, 552)	1
  (3, 1158)	1
  (3, 1170)	1
  (3, 3181)	1
  (4, 1529)	1
  :	:
  (539, 3032)	1
  (539, 3566)	1
  (540, 1140)	1
  (540, 2018)	1
  (540, 2072)	1
  (540, 3469)	1
  (541, 343)	1
  (541, 1352)	1
  (541, 3133)	1
  (541, 3488)	1
  (542, 1170)	1
  (542, 1836)	1
  (542, 2423)	1
  (542, 2746)	1
  (542, 3348)	1
  (542, 3574)	1
  (543, 1010)	1
  (543, 3572)	1
  (544, 220)	1
  (545, 1524)	1
  (545, 1625)	1
  (545, 1629)	1
  (545, 2000)	1
  (545, 2147)	1
  (545, 3483)	1


## Let's Classify some things


In [21]:
# Import classifiers

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import svm

# Import Scoring Metrics

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, roc_auc_score, make_scorer, accuracy_score, f1_score

In [22]:
# Logistic Regression

# Set Random Seed
np.random.seed(12)

X_train, X_test, y_train, y_test = train_test_split(
    X, product['Polarity'], train_size = 0.75
)

# Lets check the results
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X.shape)


(1651, 3627)
(551, 3627)
(551,)
(2202, 3627)


In [23]:
for c in [0.01, 0.05, 0.25, 0.5, 1, 2]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test, lr.predict(X_test))))
    print ("F1 C=%s: %s" 
           % (c, f1_score(y_test, lr.predict(X_test))))

Accuracy for C=0.01: 0.6642468239564429
F1 C=0.01: 0.5584725536992841
Accuracy for C=0.05: 0.7168784029038112
F1 C=0.05: 0.6694915254237288
Accuracy for C=0.25: 0.7731397459165155
F1 C=0.25: 0.7572815533980584
Accuracy for C=0.5: 0.7931034482758621
F1 C=0.5: 0.7849056603773585
Accuracy for C=1: 0.8003629764065335
F1 C=1: 0.7962962962962963
Accuracy for C=2: 0.8003629764065335
F1 C=2: 0.7970479704797048


## More Classifying

In [24]:
#Decision Tree

clf = DecisionTreeClassifier(random_state=12, 
                             min_samples_split=2, 
                             min_samples_leaf=30, 
                             max_depth=6)

clf.fit(X_train, y_train)

y_pred_dt = clf.predict(X_test)

In [25]:
confusion_matrix(y_test, y_pred_dt)

#Reporting

#Let's look at the total reporting

class_names = [str(x) for x in clf.classes_]

print(classification_report(y_test, y_pred_dt, target_names=class_names))

              precision    recall  f1-score   support

           0       0.58      0.89      0.70       272
           1       0.77      0.36      0.49       279

    accuracy                           0.62       551
   macro avg       0.67      0.63      0.60       551
weighted avg       0.67      0.62      0.59       551



In [32]:
# AdaBoost

clf2 = AdaBoostClassifier(n_estimators=200, learning_rate=0.1,random_state=2, algorithm="SAMME")
clf2.fit(X_train, y_train)

pred_val = clf2.predict(X_test)
confusion_matrix(y_test, pred_val)


print("F1 Score = {:.2f}".format(f1_score(y_test, pred_val)))
print()
print(classification_report(y_test, pred_val))

F1 Score = 0.37

              precision    recall  f1-score   support

           0       0.55      0.94      0.69       272
           1       0.81      0.24      0.37       279

    accuracy                           0.59       551
   macro avg       0.68      0.59      0.53       551
weighted avg       0.68      0.59      0.53       551



In [34]:
# SVC

clf3 = SVC(kernel='linear', random_state=42, gamma=3, degree=7)

clf3.fit(X_train, y_train)

pred_val = clf3.predict(X_test)
confusion_matrix(y_test, pred_val)


print("F1 Score = {:.2f}".format(f1_score(y_test, pred_val)))
print()
print(classification_report(y_test, pred_val))

F1 Score = 0.78

              precision    recall  f1-score   support

           0       0.76      0.80      0.78       272
           1       0.80      0.76      0.78       279

    accuracy                           0.78       551
   macro avg       0.78      0.78      0.78       551
weighted avg       0.78      0.78      0.78       551



In [36]:
# Taking another look at the Logistic Regression and play around with the solver and the penalty

clf4 = LogisticRegression(C=1, solver='liblinear', penalty='l2')
clf4.fit(X_train, y_train)
pred_val = clf4.predict(X_test)
confusion_matrix(y_test, pred_val)

print("F1 Score = {:.2f}".format(f1_score(y_test, pred_val)))
print()
print(classification_report(y_test, pred_val))
print()
print(confusion_matrix(y_test, pred_val))

F1 Score = 0.80

              precision    recall  f1-score   support

           0       0.78      0.83      0.80       272
           1       0.82      0.77      0.80       279

    accuracy                           0.80       551
   macro avg       0.80      0.80      0.80       551
weighted avg       0.80      0.80      0.80       551


[[226  46]
 [ 64 215]]


In [28]:
# Let's take a look at the predictions

for row_index, (input, prediction, label) in enumerate(zip (X_test, pred_val, product_test['Polarity'])):
  if prediction != label:
    print('Row', row_index, 'has been classified as ', prediction, 'and should be ', label)

Row 0 has been classified as  0 and should be  1
Row 1 has been classified as  0 and should be  1
Row 5 has been classified as  0 and should be  1
Row 6 has been classified as  0 and should be  1
Row 7 has been classified as  1 and should be  0
Row 8 has been classified as  0 and should be  1
Row 9 has been classified as  0 and should be  1
Row 13 has been classified as  0 and should be  1
Row 14 has been classified as  0 and should be  1
Row 16 has been classified as  1 and should be  0
Row 23 has been classified as  1 and should be  0
Row 26 has been classified as  1 and should be  0
Row 27 has been classified as  1 and should be  0
Row 28 has been classified as  1 and should be  0
Row 30 has been classified as  1 and should be  0
Row 31 has been classified as  1 and should be  0
Row 33 has been classified as  1 and should be  0
Row 34 has been classified as  1 and should be  0
Row 38 has been classified as  0 and should be  1
Row 39 has been classified as  1 and should be  0
Row 41 

Row 440 has been classified as  1 and should be  0
Row 441 has been classified as  1 and should be  0
Row 442 has been classified as  0 and should be  1
Row 443 has been classified as  0 and should be  1
Row 447 has been classified as  1 and should be  0
Row 450 has been classified as  1 and should be  0
Row 452 has been classified as  1 and should be  0
Row 453 has been classified as  1 and should be  0
Row 454 has been classified as  1 and should be  0
Row 457 has been classified as  1 and should be  0
Row 459 has been classified as  1 and should be  0
Row 461 has been classified as  0 and should be  1
Row 467 has been classified as  0 and should be  1
Row 468 has been classified as  1 and should be  0
Row 470 has been classified as  1 and should be  0
Row 472 has been classified as  0 and should be  1
Row 473 has been classified as  0 and should be  1
Row 476 has been classified as  0 and should be  1
Row 477 has been classified as  0 and should be  1
Row 482 has been classified as 

In [260]:
# Rows 0,5,13,16,23

In [29]:
a = product_test.loc[0]
b = product_test.loc[5]
c = product_test.loc[13]
d = product_test.loc[16]
e = product_test.loc[23]

In [30]:
print(a,b,c,d,e)

Sentence    A good commentary of today's love and undoubte...
Polarity                                                    1
Name: 0, dtype: object Sentence    I rather enjoyed it.  
Polarity                         1
Name: 5, dtype: object Sentence    The camera really likes her in this movie.  
Polarity                                               1
Name: 13, dtype: object Sentence    I am a fan of his ... This movie sucked really...
Polarity                                                    0
Name: 16, dtype: object Sentence    Now we were chosen to be tortured with this di...
Polarity                                                    0
Name: 23, dtype: object


In [31]:
#Let's check their polarities

from textblob import TextBlob

full = [a,b,c,d,e]

for i in full:
    tb = TextBlob(str(i))
    print(tb.sentiment)

Sentiment(polarity=0.6, subjectivity=0.6000000000000001)
Sentiment(polarity=0.5, subjectivity=0.7)
Sentiment(polarity=0.2, subjectivity=0.2)
Sentiment(polarity=0.2, subjectivity=0.2)
Sentiment(polarity=0.0, subjectivity=0.0)
